In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 开始学习使用BigQuery数据集

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_bq_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/datasets/get_started_bq_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/datasets/get_started_bq_datasets.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在生产环境中使用Vertex AI。本教程涵盖了数据管理：从BigQuery数据集开始入手。

Learn more about [BigQuery Datasets](https://cloud.google.com/bigquery/docs/datasets-intro) and [Vertex AI for BigQuery users](https://cloud.google.com/vertex-ai/docs/beginner/bqml).

### 目标

在本教程中，您将学习如何将`BigQuery`用作`Vertex AI`训练的数据集。

本教程使用以下Google Cloud ML服务：

- `Vertex AI数据集`
- `BigQuery数据集`

执行的步骤包括：

- 从`BigQuery`表创建一个`Vertex AI数据集`资源-- 适用于`AutoML`训练。
- 将数据集从`BigQuery`提取到Cloud Storage中的CSV文件-- 适用于`AutoML`或自定义训练。
- 从`BigQuery`数据集选择行到`pandas` dataframe-- 适用于自定义训练。
- 从`BigQuery`数据集选择行到`tf.data.Dataset`-- 适用于自定义训练`TensorFlow`模型。
- 从提取的CSV文件中选择行到`tf.data.Dataset`-- 适用于自定义训练`TensorFlow`模型。
- 从CSV文件创建一个`BigQuery`数据集。
- 从`BigQuery`表提取数据到`DMatrix`-- 适用于自定义训练`XGBoost`模型。

### 推荐

在Google Cloud上进行端到端（E2E）MLOps时，以下是在BigQuery中处理结构化（表格）数据时的最佳实践：

- 对于AutoML训练：
  - 使用Vertex AI的 `TabularDataset` 创建一个托管数据集。
  - 将BigQuery表用作数据集的输入。
  - 在运行AutoML训练管道作业时，指定列和列转换。


- 对于自定义训练：
  - 对于小型数据集：
    - 将BigQuery提取到一个pandas数据框中。
    - 在数据框中预处理数据。
  - 对于大型数据集：
    - TensorFlow模型训练：
      - 从BigQuery表创建一个tf.data.Dataset生成器。
      - 指定自定义训练的列。
      - 对数据进行预处理：
        - 在生成器中（上游）
        - 在模型中（下游）
    - XGBoost模型训练：
      - 使用BigQuery ML内置的XGBoost训练。
      - 或者，从BigQuery表中提取的CSV文件创建一个DMatrix生成器。
    - PyTorch模型训练：
        - 将BigQuery提取到一个pandas数据框中。
        - 在数据框中预处理数据。
        - 从pandas数据框中创建一个DataLoader生成器。


- 或者：
    - 将BigQuery表提取为CSV文件。
    - 预处理CSV文件。
    - 从CSV文件创建一个tf.data.Dataset生成器。

### 数据集

本教程使用的数据集是来自[BigQuery公共数据集](https://cloud.google.com/bigquery/public-data)的GSOD数据集。在这个数据集的版本中，您考虑年份、月份和日期字段来预测平均每日温度（mean_temp）的值。

### 成本
该教程使用 Google Cloud 的可计费组件：

- Vertex AI
- Cloud Storage
- BigQuery

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Cloud Storage 价格](https://cloud.google.com/storage/pricing) 和 [BigQuery 价格](https://cloud.google.com/bigquery/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 基于您的预计使用情况生成成本估算。

## 安装

安装以下包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-bigquery \
                                 tensorflow \
                                 tensorflow-io==0.18 \
                                 xgboost \
                                 numpy \
                                 pandas \
                                 pyarrow

### 仅限于 Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关[Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 认证您的 Google Cloud 账户

根据您的 Jupyter 环境，您可能需要手动认证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 由于您已经过身份验证，所以无需执行任何操作。

**2. 本地 JupyterLab 实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务帐号授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有如果您的存储桶尚不存在：才运行下面的单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
import pandas as pd
import xgboost as xgb
from google.cloud import bigquery

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 版的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

创建BigQuery客户端。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

请设置变量`IMPORT_FILE`为BigQuery中数据表的位置，并设置`BQ_TABLE`为表ID。

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### 创建数据集

#### BigQuery 输入数据

接下来，使用 `TabularDataset` 类的 `create` 方法创建 `Dataset` 资源，该方法接受以下参数：

- `display_name`：`Dataset` 资源的人类可读的名称。
- `bq_source`：将数据项从 BigQuery 表导入到 `Dataset` 资源中。
- `labels`：用户定义的元数据。在此示例中，您将存储包含用户定义数据的 Cloud Storage 存储桶的位置。

了解有关从 BigQuery 表创建 TabularDataset 的更多信息，请访问[此处](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python)。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data",
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_URI[5:]},
)

label_column = "mean_temp"

print(dataset.resource_name)

### 将数据集复制到云存储

接下来，您可以使用BigQuery提取命令将BigQuery表复制为CSV文件，保存到云存储中。

了解更多关于[BigQuery命令行界面](https://cloud.google.com/bigquery/docs/reference/bq-cli-reference)。

In [ ]:
comps = BQ_TABLE.split(".")
BQ_PROJECT_DATASET_TABLE = comps[0] + ":" + comps[1] + "." + comps[2]

! bq --location=us extract --destination_format CSV $BQ_PROJECT_DATASET_TABLE $BUCKET_URI/mydata*.csv

IMPORT_FILES = ! gsutil ls $BUCKET_URI/mydata*.csv

print(IMPORT_FILES)

EXAMPLE_FILE = IMPORT_FILES[0]

! gsutil cat $EXAMPLE_FILE | head

### 创建数据集

#### CSV 输入数据

接下来，使用 `TabularDataset` 类的 `create` 方法创建 `Dataset` 资源，其参数如下：

- `display_name`：`Dataset` 资源的可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入 `Dataset` 资源。
- `labels`：用户定义的元数据。在本例中，您存储包含用户定义数据的 Cloud Storage 存储桶的位置。

了解更多关于[从 CSV 文件创建 TabularDataset](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_gcs_sample-python)。

In [ ]:
gcs_source = IMPORT_FILES

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data",
    gcs_source=gcs_source,
    labels={"user_metadata": BUCKET_URI[5:]},
)


label_column = "mean_temp"

print(dataset.resource_name)

### 创建 BigQuery 数据集的视图

或者，您可以创建一个逻辑视图，其中包含 BigQuery 数据集的一部分字段。

了解更多关于[创建 BigQuery 视图](https://cloud.google.com/bigquery/docs/views)。

In [ ]:
# Set dataset name and view name in BigQuery
BQ_MY_DATASET = "[your-dataset-name]"
BQ_MY_TABLE = "[your-view-name]"

# Otherwise, use the default names
if (
    BQ_MY_DATASET == ""
    or BQ_MY_DATASET is None
    or BQ_MY_DATASET == "[your-dataset-name]"
):
    BQ_MY_DATASET = "mlops_dataset"

if BQ_MY_TABLE == "" or BQ_MY_TABLE is None or BQ_MY_TABLE == "[your-view-name]":
    BQ_MY_TABLE = "mlops_view"

In [ ]:
# Create the resources
! bq --location=US mk -d \
$PROJECT_ID:$BQ_MY_DATASET

sql_script = f'''
CREATE OR REPLACE VIEW `{PROJECT_ID}.{BQ_MY_DATASET}.{BQ_MY_TABLE}`
AS SELECT station_number,year,month,day,mean_temp FROM `{BQ_TABLE}`
'''
print(sql_script)

query = bqclient.query(sql_script)

将BigQuery数据集读入pandas dataframe中

接下来，您可以使用BigQuery的`list_rows()`和`to_dataframe()`方法，将数据集的样本读入pandas dataframe，如下所示：

- `list_rows()`: 对指定的表执行查询，并返回查询结果的行迭代器。可以选择指定：
  - `selected_fields`: 返回的字段（列）的子集。
  - `max_results`: 要返回的最大行数。与SQL LIMIT命令相同。

- `rows.to_dataframe()`: 调用行迭代器，将数据读入pandas dataframe中。

了解更多关于[将BigQuery表加载到dataframe中](https://cloud.google.com/bigquery/docs/bigquery-storage-python-pandas)。

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(BQ_TABLE)

rows = bqclient.list_rows(
    table,
    max_results=500,
    selected_fields=[
        bigquery.SchemaField("station_number", "STRING"),
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("month", "INTEGER"),
        bigquery.SchemaField("day", "INTEGER"),
        bigquery.SchemaField("mean_temp", "FLOAT"),
    ],
)

dataframe = rows.to_dataframe()
print(dataframe.head())

### 将BigQuery数据集读入tf.data.Dataset

接下来，您可以使用TensorFlow IO的`BigQueryClient()`和`read_session()`方法将数据集的样本读入tf.data.Dataset，并使用以下参数:

- `parent`: 您的项目 ID。
- `project_id`: BigQuery 表的项目 ID。
- `dataset_id`: BigQuery 数据集的 ID。
- `table_id`: 相应 BigQuery 数据集中的表的 ID。
- `selected_fields`: 要返回的字段（列）的子集。
- `output_types`: 相应字段的输出类型。
- `requested_streams`: 并行读取器的数量。

了解更多关于[BigQuery TensorFlow reader](https://www.tensorflow.org/io/tutorials/bigquery)的信息。

了解更多关于[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)的信息。

In [ ]:
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient

feature_names = "station_number,year,month,day".split(",")

target_name = "mean_temp"


def read_bigquery(project, dataset, table):
    tensorflow_io_bigquery_client = BigQueryClient()
    read_session = tensorflow_io_bigquery_client.read_session(
        parent="projects/" + PROJECT_ID,
        project_id=project,
        dataset_id=dataset,
        table_id=table,
        selected_fields=feature_names + [target_name],
        output_types=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
        requested_streams=2,
    )

    dataset = read_session.parallel_read_rows()
    return dataset


PROJECT, DATASET, TABLE = IMPORT_FILE.split("/")[-1].split(".")
tf_dataset = read_bigquery(PROJECT, DATASET, TABLE)

print(tf_dataset.take(1))

将CSV文件读入tf.data.Dataset

或者，当您的数据存储在CSV文件中时，您可以使用`tf.data.experimental.CsvDataset`将数据集加载到tf.data.Dataset中，使用以下参数：

- `filenames`：一个或多个CSV文件的列表。
- `header`：CSV文件是否包含标题。
- `select_cols`：要返回的字段（列）的子集。
- `record_defaults`：相应字段的输出类型。

了解更多关于 [tf.data CsvDataset](https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset)

In [ ]:
import tensorflow as tf

feature_names = ["station_number,year,month,day".split(",")]

target_name = "mean_temp"

tf_dataset = tf.data.experimental.CsvDataset(
    filenames=IMPORT_FILES,
    header=True,
    select_cols=feature_names.append(target_name),
    record_defaults=[dtypes.string] + [dtypes.int32] * 3 + [dtypes.float32],
)

print(tf_dataset.take(1))

### 从pandas dataframe创建一个BigQuery数据集

您可以使用BigQuery的`create_dataset()`和`load_table_from_dataframe()`方法从pandas dataframe创建一个BigQuery数据集，具体步骤如下：

- `create_dataset()`: 创建一个空的BigQuery数据集，有以下参数：
  - `dataset_ref`: 从数据集ID创建的`DatasetReference`，例如samples。
- `load_table_from_dataframe()`: 将一个或多个CSV文件加载到相应数据集中的表中，具有以下参数：
  - `dataframe`: dataframe。
  - `table`: 表格的`TableReference`。
  - `job_config`: 指定如何加载dataframe数据的规范。

In [ ]:
LOCATION = "us"

SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
]


DATASET_ID = "samples"
TABLE_ID = "gsod"


def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id)
    )
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False


def load_data_into_bigquery(dataframe, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig(
        # Specify a (partial) schema. All columns are always written to the
        # table. The schema is used to assist in data type definitions.
        schema=[
            bigquery.SchemaField("station_number", "STRING"),
            bigquery.SchemaField("year", "INTEGER"),
            bigquery.SchemaField("month", "INTEGER"),
            bigquery.SchemaField("day", "INTEGER"),
            bigquery.SchemaField("mean_temp", "FLOAT"),
        ],
        # Optionally, set the write disposition. BigQuery appends loaded rows
        # to an existing table by default, but with WRITE_TRUNCATE write
        # disposition it replaces the table with the loaded data.
        write_disposition="WRITE_TRUNCATE",
    )

    NEW_BQ_TABLE = f"{PROJECT_ID}.{dataset_id}.{table_id}"

    job = bqclient.load_table_from_dataframe(
        dataframe, NEW_BQ_TABLE, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.

    table = bqclient.get_table(NEW_BQ_TABLE)  # Make an API request.
    print(
        "Loaded {} rows and {} columns to {}".format(
            table.num_rows, len(table.schema), NEW_BQ_TABLE
        )
    )


load_data_into_bigquery(dataframe, DATASET_ID, TABLE_ID)

### 从CSV文件创建一个BigQuery数据集

您可以使用BigQuery的`create_dataset()`和`load_table_from_uri()`方法从CSV文件中创建一个BigQuery数据集，具体步骤如下：

- `create_dataset()`: 创建一个空的BigQuery数据集，使用以下参数：
  - `dataset_ref`: 从dataset_id创建的`DatasetReference`，例如samples。
- `load_table_from_uri()`: 将一个或多个CSV文件加载到相应数据集中的表中，使用以下参数：
  - `url`: 一个或多个Cloud Storage存储中的CSV文件集合。
  - `table`: 表的`TableReference`。
  - `job_config`: 有关如何加载CSV数据的规范。

了解更多关于[将CSV数据导入BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery)的信息。

In [ ]:
LOCATION = "us"

CSV_SCHEMA = [
    bigquery.SchemaField("station_number", "STRING"),
    bigquery.SchemaField("wban_number", "STRING"),
    bigquery.SchemaField("year", "INTEGER"),
    bigquery.SchemaField("month", "INTEGER"),
    bigquery.SchemaField("day", "INTEGER"),
    bigquery.SchemaField("mean_temp", "FLOAT"),
    bigquery.SchemaField("num_mean_temp_samples", "INTEGER"),
    bigquery.SchemaField("mean_dew_point", "FLOAT"),
    bigquery.SchemaField("num_mean_dew_point_samples", "INTEGER"),
    bigquery.SchemaField("mean_sealevel_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_sealevel_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_station_pressure", "FLOAT"),
    bigquery.SchemaField("num_mean_station_pressure_samples", "INTEGER"),
    bigquery.SchemaField("mean_visibility", "FLOAT"),
    bigquery.SchemaField("num_mean_visibility_samples", "INTEGER"),
    bigquery.SchemaField("mean_wind_speed", "FLOAT"),
    bigquery.SchemaField("num_mean_wind_speed_samples", "INTEGER"),
    bigquery.SchemaField("max_sustained_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_gust_wind_speed", "FLOAT"),
    bigquery.SchemaField("max_temperature", "FLOAT"),
    bigquery.SchemaField("max_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("min_temperature", "FLOAT"),
    bigquery.SchemaField("min_temperature_explicit", "BOOLEAN"),
    bigquery.SchemaField("total_percipitation", "FLOAT"),
    bigquery.SchemaField("snow_depth", "FLOAT"),
    bigquery.SchemaField("fog", "BOOLEAN"),
    bigquery.SchemaField("rain", "BOOLEAN"),
    bigquery.SchemaField("snow", "BOOLEAN"),
    bigquery.SchemaField("hail", "BOOLEAN"),
    bigquery.SchemaField("thunder", "BOOLEAN"),
    bigquery.SchemaField("tornado", "BOOLEAN"),
]


DATASET_ID = "samples"
TABLE_ID = "gsod"


def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1  # heading

    load_job = bqclient.load_table_from_uri(url, table, job_config=job_config)
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table)
    print("Loaded {} rows.".format(destination_table.num_rows))


load_data_into_bigquery(IMPORT_FILES, DATASET_ID, TABLE_ID)

### 将BigQuery表读入XGboost DMatrix

目前，BigQuery和开源XGBoost之间没有直接的数据传输连接。BigQuery ML服务具有内置的XGBoost训练模块。

或者，您可以将数据提取为pandas dataframe或CSV文件。提取的数据然后在训练模型时作为`DMatrix`对象的输入。

了解更多关于[如何开始使用内置XGBoost](https://cloud.google.com/ai-platform/training/docs/algorithms/xgboost-start)。

### 将pandas表读入XGboost DMatrix

接下来，您将pandas数据框加载到`DMatrix`对象中。XGBoost不支持非数值输入。在加载数据框之前，任何分类列都需要进行独热编码。

In [ ]:
dataframe["station_number"] = pd.to_numeric(dataframe["station_number"])
labels = dataframe["mean_temp"]
data = dataframe.drop(["mean_temp"], axis=1)

dtrain = xgb.DMatrix(data, label=labels)

### 将CSV文件读入XGboost DMatrix

目前，XGBoost不支持云存储。如果要使用CSV文件作为输入，您需要将它们下载到本地。

In [ ]:
! gsutil cp $EXAMPLE_FILE data.csv

dtrain = xgb.DMatrix("data.csv?format=csv&label_column=4")

清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除此教程中创建的各个资源：

- Vertex AI数据集资源
- 云存储桶
- BigQuery数据集

将`delete_storage`设置为_True_以删除在此笔记本中使用的存储资源。

In [ ]:
import os

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the temporary BigQuery dataset
! bq rm -r -f $PROJECT_ID:$DATASET_ID

delete_storage = False
if delete_storage or os.getenv("IS_TESTING"):
    # Delete the created GCS bucket
    ! gsutil rm -r $BUCKET_URI
    # Delete the created BigQuery datasets
    ! bq rm -r -f $PROJECT_ID:$BQ_MY_DATASET